In [127]:
# label / target preprocessing / splitting
import numpy as np
from random import shuffle

n_points = 2000
shuffle_data = True

def target_four_hot_decoder(n_points):
    # four_hot encoder
    labels = []
    
    for i in range(1,n_points+1):
        filename = './simple_minion_dataset/targets/target%i.txt' % (i)
        temp = np.array([0,0,  0,0,0,0,0,0,0,0,0,0,  0,0,0,0,0,0,0,0,0,  0,0,0,0,0,0,0,0,0,0])
        with open(filename, 'r') as f:
            for line in f:
                items = line.split(':')
                key, value = items[0], float(items[1])
                if key == "Eyes":
                    temp[int(value)] = 1
                elif key == "Height":
                    temp[int(value*10+1)] = 1
                elif key == "Arm_Length":
                    temp[int(value*10+10)] = 1
                elif key == "Leg_Length":
                    temp[int(value*10+20)] = 1
        labels.append(temp)

    return labels

def taget_four_units_decoder(n_points):
    #decodes targets as the floating point values they are
    labels = []
    
    for i in range(1,n_points+1):
        filename = './simple_minion_dataset/targets/target%i.txt' % (i)
        temp = np.array([])
        with open(filename, 'r') as f:
            for line in f:
                items = line.split(':')
                key, value = items[0], float(items[1])
                temp = np.append(temp, value)
        labels.append(temp)

    return labels


minion_files = [("./simple_minion_dataset/data/minion%d.png" % i) for i in range(1,n_points+1)]
minion_targets = taget_four_units_decoder(n_points)
        
# shuffle data if desired
if shuffle_data:
    packed = list(zip(minion_files, minion_targets))
    shuffle(packed)
    minion_files, minion_targets = zip(*packed)
    
# devide data into 60:20:20
train_minion_files = minion_files[0:int(0.6*len(minion_files))]
train_minion_targets = minion_targets[0:int(0.6*len(minion_targets))]

val_minion_files = minion_files[int(0.6*len(minion_files)):int(0.8*len(minion_files))]
val_minion_targets = minion_targets[int(0.6*len(minion_targets)):int(0.8*len(minion_targets))]

test_minion_files = minion_files[int(0.8*len(minion_files)):]
test_minion_targets = minion_targets[int(0.8*len(minion_targets)):]
    

                
                


In [128]:
import tensorflow as tf
from tensorflow.contrib.data import Dataset, Iterator
from itertools import count
from datetime import datetime

tf.reset_default_graph()

BatchSize = 10  # tf.constant(10, dtype=tf.int64)
Epochs = 100  # tf.constant(50, dtype=tf.int64)
Height = 100
Width = 100
Channels = 3
Validation_Size = len(val_minion_files)

eyes_accuracy = []
height_accuracy = []
arms_accuracy = []
legs_accuracy = []

def read_from_file(filename, target):
    file_content = tf.read_file(filename)
    decoded = tf.image.decode_png(file_content, channels=3, dtype=tf.uint8)
    normalised = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize_images(normalised, (Height, Width))
    
    return resized, target


# Get training dataset into TensorFlow
minion_train = Dataset.from_tensor_slices((
    tf.convert_to_tensor(train_minion_files), 
    tf.convert_to_tensor(np.array(train_minion_targets).astype(np.float32))
    ))

minion_train = minion_train.map(read_from_file, num_threads=2, output_buffer_size=200)
minion_train = minion_train.shuffle(buffer_size=20000) # another way to shuffle data
minion_train = minion_train.batch(BatchSize)
minion_train = minion_train.repeat(Epochs)

# Get test dataset into TesorFlow
minion_test = Dataset.from_tensor_slices((
    tf.convert_to_tensor(test_minion_files),
    tf.convert_to_tensor(np.array(test_minion_targets).astype(np.float32))
    ))

minion_test = minion_test.map(read_from_file, num_threads=2, output_buffer_size=200)
minion_test = minion_test.shuffle(buffer_size=20000)
minion_test = minion_test.batch(BatchSize)
minion_test = minion_test.repeat(Epochs)

# Get validation dataset into TesorFlow
minion_val = Dataset.from_tensor_slices((
    tf.convert_to_tensor(val_minion_files),
    tf.convert_to_tensor(np.array(val_minion_targets).astype(np.float32))
    ))

minion_val = minion_val.map(read_from_file, num_threads=2, output_buffer_size=200)
# minion_val = minion_val.shuffle(buffer_size=20000) # no shuffle neccessary
minion_val = minion_val.batch(Validation_Size)
minion_val = minion_val.repeat(1) # no repeat necessary

train_iterator_handle = minion_train.make_one_shot_iterator().string_handle()
test_iterator_handle  = minion_test.make_one_shot_iterator().string_handle()
val_iterator_handle   = minion_val.make_one_shot_iterator().string_handle()

# Train and Test interators and handles
dataset_handle          = tf.placeholder(tf.string, shape=[])
iterator                = Iterator.from_string_handle(dataset_handle, (tf.float32,tf.float32), ((None, 100, 100, 3), (None, 4)))
imagebatch, targetbatch = iterator.get_next()

## Net Construction

# first conv_relu layer
h1 = tf.layers.conv2d(imagebatch, filters=32, kernel_size=5, strides=(4,4), 
                    activation=tf.nn.relu)
# second conv_relu layer
h2 = tf.layers.conv2d(h1, filters=64, kernel_size=2, strides=(2,2), 
                    activation=tf.nn.relu)
# flatten for dense
h2_flat = tf.contrib.layers.flatten(h2)

# first dense layer
h3 = tf.layers.dense(h2_flat, units=512, activation=tf.nn.tanh)

# add dropout
keep_prob = tf.placeholder(tf.float32, shape=[])
h3_drop = tf.nn.dropout(h3, keep_prob)

# dense output layer
net_out = tf.layers.dense(h3_drop, units=4, activation=tf.nn.sigmoid)

loss = tf.losses.mean_squared_error(targetbatch, net_out)
tf.summary.scalar('MSE_loss', loss)
train_step = tf.train.AdamOptimizer().minimize(loss)

merged = tf.summary.merge_all()




# Training Session
with tf.Session() as sess:
    time = datetime.now().strftime('%y%m%d-%H%M%S')    
    train_writer      = tf.summary.FileWriter('Summaries/%s/train' % time, sess.graph)
    test_writer       = tf.summary.FileWriter('Summaries/%s/test' % time)
    validation_writer = tf.summary.FileWriter('Summaries/%s/validation' % time)
    
    tf.global_variables_initializer().run()
    
    trainhandle_string = sess.run(train_iterator_handle) # evaluate to accepted handla format
    testhandle_string  = sess.run(test_iterator_handle)
    valhandle_string   = sess.run(val_iterator_handle)
    for i in count():
        try:
            _ = sess.run( train_step, {dataset_handle: trainhandle_string, keep_prob: 0.5})
        except tf.errors.OutOfRangeError:
            break
        
        if not i % 200:
            train_summary = sess.run(merged, {dataset_handle: trainhandle_string, keep_prob: 1.0})
            train_writer.add_summary(train_summary, i)
            test_summary, net_out_batch, target_batch = sess.run([merged, net_out, targetbatch], {dataset_handle: testhandle_string, keep_prob: 1.0})
            test_writer.add_summary(test_summary, i)
            
            
            
            eyes   = np.equal(np.round(net_out_batch[:,0]), target_batch[:,0])
            eyes_accuracy.append(np.mean(eyes))
            height = np.equal(np.round(net_out_batch[:,1],1), target_batch[:,1])
            height_accuracy.append(np.mean(height))
            arms   = np.equal(np.round(net_out_batch[:,2],1), target_batch[:,2])
            arms_accuracy.append(np.mean(arms))
            legs   = np.equal(np.round(net_out_batch[:,3],1), target_batch[:,3])
            legs_accuracy.append(np.mean(legs))
            
    validation_summary, val_net_out, val_target_batch = sess.run([merged, net_out, targetbatch], {dataset_handle: valhandle_string, keep_prob: 1.0})
    validation_writer.add_summary(validation_summary, i)


In [129]:
# batchwise performance on testing data during training on eyes, height, arm_length, leg_length
batch_performance = np.transpose([eyes_accuracy,
                                     height_accuracy,
                                     arms_accuracy,
                                     legs_accuracy])

print('Featurewise performance during training of the ANN on test data batches: ')
print(batch_performance)
print('Number of tests: %d' % batch_performance.shape[0])

val_net_out_round = np.concatenate((np.expand_dims(np.round(val_net_out[:,0]).T,axis=1),np.round(val_net_out[:,1:],1)), axis=1)
val_results = np.equal(val_net_out_round, val_target_batch)

val_results_total = np.all(val_results, axis=1)
validation_performance = np.mean(val_results_total)

validation_performance_featurewise = np.mean(val_results, axis=0)

print('Performance on validation data: %d %%' % (validation_performance*100))

print('Featurewise performance on validation data: ')
print('eyes, height, arm length, leg length')
print(validation_performance_featurewise*100)

Featurewise performance during training of the ANN on test data batches: 
[[ 0.6  0.   0.   0.2]
 [ 1.   1.   0.9  0.7]
 [ 1.   1.   1.   0.6]
 [ 1.   1.   1.   0.5]
 [ 1.   1.   0.9  0.6]
 [ 1.   1.   0.9  1. ]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   0.8]
 [ 1.   1.   1.   0.8]
 [ 1.   1.   1.   0.7]
 [ 1.   1.   1.   0.6]
 [ 1.   1.   1.   0.9]
 [ 1.   1.   1.   0.9]
 [ 1.   1.   1.   0.9]
 [ 1.   1.   1.   0.7]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   0.9]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   0.7]
 [ 1.   1.   1.   0.7]
 [ 1.   1.   1.   0.8]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   0.9]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   0.9]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   0.8]
 [ 1.   0.9  0.9  1. ]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   0.9]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   0.8]
 [ 1.   1.   1.   0.9]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   1. ]
 [ 1.   1.   1.   0.9]
 [ 1. 